In [104]:
#some ideas from Fayem Hathak
import numpy as np
import scipy.io.wavfile 
import math
import sys
from scipy.fftpack import dct

In [106]:
first_file = 'digits/1a.wav'
(sample_rate,signal) = scipy.io.wavfile.read(first_file)
signal

array([2, 2, 0, ..., 2, 1, 2], dtype=int16)

In [123]:
# take 256 points FFT
window_size = 256
# shift = 20
#pre_emphasis

# pre-Emphasis
alpha = 0.97 #preemphais coeff
emphasized_signal = np.append(signal[0], signal[1:] - alpha * signal[:-1])
# hamming = np.hamming(window_size)# hamming window 
# window_start = np.arange(0,new_signal.shape[0]-window_size,shift) # position of the start of the window
# # windowing
# frames = np.zeros([window_start.shape[-1],window_size],dtype = complex)
# # frame
# for i in np.arange(0,window_start.shape[-1]):
#     start = window_start[i]
#     X = np.fft.fft(new_signal[start:(start+window_size)]*hamming)
#     frames[i,:] = X
frame_size = 0.025
frame_stride = 0.01


frame_length, frame_step = frame_size * sample_rate, frame_stride * sample_rate  # Convert from seconds to samples
signal_length = len(emphasized_signal)# length of the signals
frame_length = int(round(frame_length)) # round the frame_length
frame_step = int(round(frame_step)) # get the step of the frame
num_frames = int(np.ceil(float(np.abs(signal_length - frame_length)) / frame_step))  # Make sure that we have at least 1 frame
pad_signal_length = num_frames * frame_step + frame_length
z = np.zeros((pad_signal_length - signal_length))
pad_signal = np.append(emphasized_signal, z) # Pad Signal to make sure that all frames have equal number of samples without truncating any samples from the original signal
indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).transpose()
frames = pad_signal[indices.astype(np.int32, copy=False)]

frames *= np.hamming(frame_length)
mag_frames = np.absolute(np.fft.rfft(frames, window_size))
pow_frames = ((1.0 / window_size) * ((mag_frames) ** 2))
# for i  in wi
# we did not have duplicates
high_mel_freq = 1125 * np.log(1 + (sample_rate/2) / 700)
low_mel_freq = 0
num_filter = 40 # number of filters
mel_points = np.linspace(low_mel_freq, high_mel_freq, num_filter + 2)
# in hertz
hz_points = 700 * (math.e**(mel_points / 1127) - 1)
bin = np.floor((window_size + 1) * hz_points / sample_rate)
# filter banks
fbank = np.zeros((num_filter, int(np.floor(window_size/2 + 1))))
for m in range(1, num_filter+1):
    f_m_left = int(bin[m - 1])    # left
    f_m = int(bin[m])             # center
    f_m_right = int(bin[m + 1])   # right
    for k in range(f_m_left, f_m):
        fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1]) #implement the filtering
    for k in range(f_m, f_m_right):
        fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m]) #implement the filtering
# dot product between each bank with the power sum of the frame       
filter_banks = np.dot(pow_frames, fbank.transpose()) # find the forier  
filter_banks = np.where(filter_banks == 0, np.finfo(float).eps, filter_banks)  # Numerical Stability
filter_banks = 20 * np.log(filter_banks) # take the logs
# filter_banks
num_ceps = 12 #number of ceps coefficients
filter_banks

array([[ -45.92248052,  -27.85801638,  -18.94510424, ...,   48.56659859,
          47.25829945,   55.27438165],
       [ -53.36830903,  -39.31915389,  -21.72457235, ...,   33.38367164,
          43.75532976,   48.86647684],
       [-106.20967158,  -38.78208066,  -51.79248454, ...,   45.60205284,
          50.17865609,   47.1292982 ],
       ...,
       [  -5.42218738,   -3.73613786,  -29.53261325, ...,   51.58408122,
          57.34940589,   56.76356199],
       [  -1.33646404,  -70.18774845,  -30.34573704, ...,   40.6958418 ,
          45.28948928,   53.53474934],
       [  38.15758681,   25.77228196, -124.74219861, ...,   21.01390259,
          38.23031592,   52.34271163]])

In [108]:
# extract the 12 features
mfcc = dct(filter_banks, type=2, axis=1, norm='ortho')[:, 1 : (num_ceps + 1)] # keep ceps from 2 to 13

In [116]:
np.shape(mfcc)

(74, 12)

In [120]:
np.dot(mfcc[1,:],mfcc[1,:].T)

221295.24926278047

In [122]:
len(mfcc)

74